# Yield prediction exploration

Goal of this exploration is to find a model that can predict the yield of a PV-system based on:

- installation data
- weather data

## Required libraries

- [pandas](https://pypi.org/project/pandas/)
- [pgeocode](https://pypi.org/project/pgeocode/)
- [geopandas](https://pypi.org/project/geopandas/)
- [matplotlib](https://pypi.org/project/matplotlib/)
- [scikit-learn](https://pypi.org/project/scikit-learn/)


In [ ]:
%pip install pandas
%pip install pgeocode
%pip install geopandas
%pip install matplotlib
%pip install scikit-learn
%pip install openpyxl
%pip install geopy
%pip install suncalc

In [ ]:
import pandas as pd
import pgeocode
from geopy import distance
import geopandas as gpd
import matplotlib.pyplot as plt

## Basic setup

To make things easier, we will first define some constants and functions that we will use throughout this notebook.

### Constants

- `DATA_DIR`: directory where the data is stored
- `GENERAL_INSTALLATION_DATA`: file path of the general installation data
- `GENERAL_WEATHER_STATION_DATA`: file path of the general weather station data
- `INSTALLATION_YIELD_DATA`: file path of the installation yield data
- `WEATHER_STATION_DATA`: file path of the weather station data


In [ ]:
import os

# Constants

DATA_DIR = os.path.join("..", "data")
GENERAL_INSTALLATION_DATA = os.path.join(DATA_DIR, "general_installation_data.xlsx")
GENERAL_WEATHER_STATION_DATA = os.path.join(DATA_DIR, "general_weather_station_data.xlsx")
INSTALLATION_YIELD_DATA = os.path.join(DATA_DIR, "installation_yield_data.xlsx")
WEATHER_STATION_DATA = os.path.join(DATA_DIR, "weather_station_data.xlsx")

NETHERLANDS_MAP_DATA = os.path.join(DATA_DIR, "map", "gadm41_NLD_2.shp")

DATA_SET = os.path.join(DATA_DIR, "dataset.csv")

## Data exploration

### General installation data

In [ ]:
general_installation_data = pd.read_excel(GENERAL_INSTALLATION_DATA)

general_installation_data.head()

Looking at the first 5 rows of the general installation data already see that we have quite installation data available. The most interesting columns right now are:

- `id`: unique identifier of the installation
- `zipcode`: zipcode of the installation
- `weather_station_id`: id of the weather station that is closest to the installation
- `geometric_yield`: UNKNOWN
- `orientation_deg_north`: orientation of the installation in degrees
- `roof_inclination`: inclination of the installation in degrees

We also see a few columns that look like they are either a constant (e.g. `efficiency`), or can be calculated based on other columns (e.g. `total_output` as `number_of_panels` * `output`). Lets validate if this assumptions are correct and remove the columns that are not needed.

In [ ]:
general_installation_data["efficiency"].value_counts()

Looking at the efficiency column, we see that it is indeed a constant. We can remove this column since its not relevant for our prediction.

In [ ]:
general_installation_data.drop(columns=["efficiency"], inplace=True)

We also have a column that is not named (`Unnamed: 19`), lets see, if this column contains any useful information.

In [ ]:
general_installation_data["Unnamed: 19"].value_counts()

Since this is also a constant, we can just drop it.

In [ ]:
general_installation_data.drop(columns=["Unnamed: 19"], inplace=True)

One of the columns that looked like its calculated was the `total_output` column. Lets see if this is true.

In [ ]:
calculated_total_output = general_installation_data["number_of_panels"] * general_installation_data["output"]

calculated_total_output.equals(general_installation_data["total_output"])

Indeed, the total output can be calculated based on the number of panels and the output per panel. We can remove this column since its not relevant for our prediction.

In [ ]:
general_installation_data.drop(columns=["total_output"], inplace=True)

In [ ]:
calculated_total_inverter_output = general_installation_data["amount"] * general_installation_data["power"]

calculated_total_inverter_output.equals(general_installation_data["total_inverter_output"])

Also the `total_inverter_output` is just a calcualted column. We can remove this column since its not relevant for our prediction.

In [ ]:
general_installation_data.drop(columns=["total_inverter_output"], inplace=True)

Now that we have only the interesting columns left, lets take a look at the data to see, if we need to do some more cleaning.

In [ ]:
general_installation_data.isnull().any()

Looks like we have no missing values in the columns that we are interested in.

Lets take a look at the columns and see, if we can derive some more information from them (do some feature engineering).

The `zipcode` column looks like it contains the zipcode of the installation. Since it is a categorial value, we would have to create dummies for each zipcode. This would result in a lot of columns. We can use the `pgeocode` library to get the latitude and longitude of the zipcode. This would result in 2 new columns, which is much better.

In [ ]:
nomi = pgeocode.Nominatim('nl')

# Get the location from a postcode
def get_location_from_postcode(postcode: str):

    code = postcode[:4]

    result =  nomi.query_postal_code(code)

    return pd.Series({
        "latitude": result.latitude,
        "longitude": result.longitude,
    })
    
installation_locations = general_installation_data.apply(lambda x: get_location_from_postcode(x['zipcode']), axis=1)

general_installation_data = pd.concat([general_installation_data, installation_locations], axis=1)

general_installation_data.head()

Even though the location is not exactly the location of the installation, it should be close enough for our prediction.

Since we have the latitude and longitude of the installation, we can now remove the `zipcode` column and take a look at where the installations are located.

In [ ]:
general_installation_data.drop(columns=["zipcode"], inplace=True)

In [ ]:
netherlands_map = gpd.read_file(NETHERLANDS_MAP_DATA)

general_installation_data_locations = gpd.GeoDataFrame(
    general_installation_data, 
    geometry=gpd.points_from_xy(
        general_installation_data["longitude"], 
        general_installation_data["latitude"]
        )
    )


fig, ax = plt.subplots()
netherlands_map.plot(ax=ax, color='lightgrey')
general_installation_data_locations.plot(ax=ax, color='red', marker='o', markersize=10)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Installation locations in the Netherlands")
plt.savefig("installation_locations.png")
plt.show()

## General weather station data

In [ ]:
general_weather_station_data = pd.read_excel(GENERAL_WEATHER_STATION_DATA)

general_weather_station_data.head()

Looking at the first 5 rows of the general weather station data we see, that we have not a lot of data here. But since we have the latitude and longitude of the weather stations, we can plot them on a map together with the installations.

In [ ]:
general_weather_station_data_locations = gpd.GeoDataFrame(
    general_weather_station_data, 
    geometry=gpd.points_from_xy(
        general_weather_station_data["lon"], 
        general_weather_station_data["lat"]
        )
    )


fig, ax = plt.subplots()
netherlands_map.plot(ax=ax, color='lightgrey')
general_installation_data_locations.plot(ax=ax, color='gray', marker='o', markersize=10)
general_weather_station_data_locations.plot(ax=ax, color='blue', marker='x', markersize=10)
plt.xlabel("Longitude")
plt.ylabel("Latitude")
plt.title("Installation & weather station locations in the Netherlands")
plt.savefig("installation_weather_station_locations.png")
plt.show()

Since the weather station data and the installation data both have latitude and longitude, we can calculate the distance between them to get a new feature that might be useful for our prediction.

In [ ]:
def calculate_distance_to_weather_station(weather_station_id: str, latitude: float, longitude: float):
    weather_station = general_weather_station_data[general_weather_station_data["id"] == weather_station_id].iloc[0]

    return distance.distance([latitude, longitude], [weather_station["lat"], weather_station["lon"]]).m


general_installation_data["weather_station_distance_m"] = general_installation_data_locations.apply(
    lambda x: calculate_distance_to_weather_station(x["weather_station_id"], x["latitude"], x["longitude"]), axis=1
)

general_installation_data.head()


## Weather station data

In [ ]:
weather_station_data = pd.read_excel(WEATHER_STATION_DATA)

weather_station_data.head()

Looking at the data, we have quite some cleaning to do. Lets first transform the `time_start` into a datetime object and create a new column `date` that contains only the date.

In [ ]:
weather_station_data["date"] = pd.to_datetime(weather_station_data["time_start"], format="%Y-%m-%d")

weather_station_data.drop(columns=["time_start"], inplace=True)

When we look at the temperature, we see that this can not be °C. It seems like the temperature has been multiplied by 10. Lets fix this.

In [ ]:
weather_station_data["min_temperature"] = weather_station_data["min_temperature"] / 10
weather_station_data["max_temperature"] = weather_station_data["max_temperature"] / 10
weather_station_data["average_temperature"] = weather_station_data["average_temperature"] / 10

Lets see if we have any missing values in the data.

In [ ]:
weather_station_data.isnull().any()

Some of the entries do not have any values for the weather. We have two options now:

1. Remove the entries without weather data and don't train the model on these entries
2. Try to come up with a way to fill the missing values

We will for now just remove the entries without weather data.

In [ ]:
weather_station_data.dropna(inplace=True)

weather_station_data.isnull().any()

Now that we have only entries with weather data left, lets take a look at the installation yield data.

## Installation yield data

In [ ]:
installation_yield_data = pd.read_excel(INSTALLATION_YIELD_DATA)

installation_yield_data.head()

The installation yield data requires quite some cleaning, lets go for it. Lets start by renaming the columns to something more meaningful.

In [ ]:
installation_yield_data.rename(
    columns={
        "Day of Periode 2": "date", 
        "Unnamed: 3": "measured_yield"
    }, 
    inplace=True
    )

Now lets transform the `date` column into a datetime object.

In [ ]:
installation_yield_data["date"] = pd.to_datetime(installation_yield_data["date"], format="%Y-%m-%d")

As always, lets check for missing values. In this case, we can directly drop the rows with missing values since it does not make sense to somehow try to fill the missing values.

In [ ]:
installation_yield_data.dropna(inplace=True)

The data now is ready and we can start creating the dataset that we will use for training the model.

## Create dataset

In the steps above, we have cleaned the data and created some new features. Now we can create the dataset that we will use for training the model.

To do so, we will merge all cleaned dataframes together. This can be done using the `id` column and the `weather_station_id` + `date` columns.

In [ ]:
dataset = pd.merge(
    installation_yield_data, 
    general_installation_data, 
    how="left", 
    left_on=["id"], 
    right_on=["id"]
    ).merge(
        weather_station_data,
        how="left",
        left_on=["weather_station_id", "date"],
        right_on=["weather_station_id", "date"],
        suffixes=("", "_weather")
    )

dataset_index = pd.MultiIndex.from_frame(
    dataset[["id", "date"]],
    names=["id", "date"]
)

dataset.set_index(dataset_index, inplace=True)

dataset.dropna(inplace=True)

dataset.info()

Since our model is not able to handle categorial values, we will create dummies for the columns containing categorial values we want to use and remove all categorial columns from the dataset afterwards.

In [ ]:
dataset = pd.get_dummies(dataset, columns=["panel_brand", "panel_model", "inverter_brand", "inverter_model"])

dataset.drop(columns=["id", "pchn_x", "pchn_y", "weather_station_id", "name"], inplace=True)

Now there is only one column left that can not be used for training our model: the `date` column. But this column holds some very valuable data. For this reason, we need to do some form of feature engineering to make this data usable for our model.

__Why the data is interesting?__

A key factor that determines the yield of the PV-system is the angle of the sun rays hitting the panels. The angle of the sun rays depends on the location, the time of the day and the time of the year. Since we have the date of the measurement, we can calculate the angle of the sun rays hitting the panels. After calculating the angle, we can remove the `date` column as well as the `latitude` and `longitude` columns and the `geometric_yield`, `orientation_deg_north` and `roof_inclination` columns.

In [ ]:
from suncalc import get_position, get_times
from datetime import datetime, timedelta

def calculate_naive_average_sun_position(year: int, month: int, day: int, latitude: float, longitude: float):
    times = get_times(date=datetime(year, month, day, 0, 0, 0), lng=longitude, lat=latitude)

    positions = []

    time = times['sunrise']
    end = times['sunset']

    azimuth_sum = 0
    altitude_sum = 0
    num_samples = 0

    while time <= end:
        position = get_position(date=time, lng=longitude, lat=latitude)

        azimuth = position['azimuth']
        altitude = position['altitude']

        positions.append({"datetime": time, "azimuth": azimuth, "altitude": altitude})

        azimuth_sum += azimuth
        altitude_sum += altitude
        num_samples += 1

        time += timedelta(minutes=10)

    average_azimuth = azimuth_sum / num_samples
    average_altitude = altitude_sum / num_samples

    return pd.Series({
        "average_azimuth": average_azimuth,
        "average_altitude": average_altitude,
    })

naive_sun_position = dataset.apply(
    lambda x: calculate_naive_average_sun_position(
        year=x["date"].year, 
        month=x["date"].month,
        day=x["date"].day,
        latitude=x["latitude"], 
        longitude=x["longitude"]
    ), axis=1)

dataset = pd.concat([dataset, naive_sun_position], axis=1)

Not that we have translated the date into the average sun position, we can remove the `date` column.

In [ ]:
dataset.drop(columns=["date"], inplace=True)

Now that we have dropped the `date` column, we should have only numerical columns left. Lets check if this is true.

In [ ]:
dataset.info()

Looks good! We only have bools, floats and integers left. The dataset is now ready to be used for training the model.

Since the preprocessing process takes quite some time, we want to save the cleaned dataset to a file. This way, we can just load the cleaned dataset from the file instead of having to run the preprocessing process every time we want to train the model.

In [ ]:
dataset.to_csv(DATA_SET, index=True)

# Model training

With the now cleaned data, we can start training a model. We will use the [MLPRegressor](https://scikit-learn.org/stable/modules/generated/sklearn.neural_network.MLPRegressor.html) from [scikit-learn](https://scikit-learn.org/stable/index.html). To identify the best parameters for the model, we will use [GridSearch](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.GridSearchCV.html) to try out different combinations of parameters.

## Create training and test data

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

dataset = pd.read_csv(DATA_SET, parse_dates=["date"])

dataset_index = pd.MultiIndex.from_frame(
    dataset[["id", "date"]],
    names=["id", "date"]
)

dataset.set_index(dataset_index, inplace=True)
dataset.drop(columns=["id", "date"], inplace=True)

dataset.dropna(inplace=True)

X_scaler = StandardScaler()
X = dataset.drop(columns=["measured_yield"])
X = X_scaler.fit_transform(X)

y_scaler = StandardScaler()
y = dataset["measured_yield"]
y = y_scaler.fit_transform(y.values.reshape(-1, 1))

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


Now that we have out train and test data split, we have to define the possible parameters for the model. GridSearch will then try out all possible combinations of the parameters and return the best combination.

In [ ]:
param_grid = [
    {
        "activation": ["logistic", "tanh", "relu"],
        "solver": ["lbfgs", "sgd", "adam"],
        "max_iter": [1_000, 2_000, 3_000, 5_000, 10_000],
        "hidden_layer_sizes": [
            (1,),
            (2,),
            (4,),
            (8,),
            (16,),
            (32,),
            (64,),
        ],
    }
]

Having the parameters defined, we can now run the GridSearch and see what the best parameters are.

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPRegressor

grid_search = GridSearchCV(
    estimator=MLPRegressor(),
    param_grid=param_grid,
    verbose=2,
    n_jobs=5
)

grid_search.fit(X_train, y_train)

After running the GridSearch, we now want to save the result to a file so that we can load them later on.

In [ ]:
with open("best_params.txt", "w") as f:
    f.write(str(grid_search.best_params_))

with open("best_estimator.txt", "w") as f:
    f.write(str(grid_search.best_estimator_))

with open("best_score.txt", "w") as f:
    f.write(str(grid_search.best_score_))

with open("cv_results.txt", "w") as f:
    f.write(str(grid_search.cv_results_))

Having the best parameters identified, we can now train the model with the best parameters.

In [ ]:
from sklearn.neural_network import MLPRegressor

with open("best_params.txt", mode="r") as f:
        best_params = eval(f.read())


model = MLPRegressor(
    activation=best_params["activation"],
    hidden_layer_sizes=best_params["hidden_layer_sizes"],
    max_iter=best_params["max_iter"],
    solver=best_params["solver"],
    verbose=True,
)

model.fit(X_train, y_train)

Because training takes quite a while, we want to save the trained model to a file so that we can load it later on.

In [ ]:
import joblib

joblib.dump(model, "model.joblib")

To see how the model performs, we will use the test data to predict the yield and compare it to the actual yield.

In [ ]:
from sklearn.metrics import mean_squared_error
import joblib

model = joblib.load("model.joblib")

y_pred = model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)

f"Root mean squared error: {rmse}"

To get a better understanding of how the model performs, we will plot the predicted yield vs the actual yield vs the calculated yield (using the formula that Wocozon is using right now).

In [ ]:
y_pred = model.predict(X)

rmse = mean_squared_error(y, y_pred, squared=False)

f"Root mean squared error: {rmse}"

In [ ]:
def calculate_yield(
    global_solar_radiation: float,
    number_of_panels: int,
    output: float,
    efficiency: float,
    geometric_yield: float,
    radiation_freedom: float,
):
    return round((global_solar_radiation/360 * number_of_panels * output * efficiency * geometric_yield * radiation_freedom)/1000, 2)

y_calculated = dataset.apply(
    lambda x: calculate_yield(
        global_solar_radiation=x["global_solar_radiation"],
        number_of_panels=x["number_of_panels"],
        output=x["output"],
        efficiency=0.86, # `efficiency` was one of the columns in the original dataset, but it was always 0.86
        geometric_yield=x["geometric_yield"],
        radiation_freedom=x["radiation_freedom"],
    ),
    axis=1
)

rmse = mean_squared_error(dataset["measured_yield"], y_calculated, squared=False)

f"Root mean squared error: {rmse}"

Having the measured yield, the predicted yield and the calculated yield, we can now visualize the difference between them. This requires some work, because we have scaled the data before training the model. We need to reverse the scaling to get the actual values.

In [ ]:
y_pred = y_scaler.inverse_transform(y_pred.reshape(-1, 1))

dataset["predicted_yield"] = y_pred
dataset["calculated_yield"] = y_calculated

In [ ]:
unique_ids = dataset.index.get_level_values("id").unique()

unique_ids = unique_ids[:10]

fig, axes = plt.subplots(nrows=len(unique_ids), figsize=(40, 20), sharex=True)

for index, id in enumerate(unique_ids):
    data = dataset.loc[id]

    axes[index].plot(data.index.get_level_values("date"), data["measured_yield"], label="Measured yield")
    axes[index].plot(data.index.get_level_values("date"), data["calculated_yield"], label="Calculated yield")
    axes[index].plot(data.index.get_level_values("date"), data["predicted_yield"], label="Predicted yield")
    axes[index].set_title(f"ID: {id}")

plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig("plots.png")

Looking at the plot, the resuts look very promising. The predicted yield is very close to the measured yield and far more precise than the calculated yield.

In [ ]:
from IPython.display import Image, display

display(Image(filename='./plots.png', embed=True))

Probably the predictions can be improved even more my tweaking the parameters or by using a different model. But for now, this is good enough.

In [ ]:
fig, axes = plt.subplots(nrows=len(unique_ids), figsize=(40, 20), sharex=True)

for index, id in enumerate(unique_ids):
    data = dataset.loc[id]

    measured_yield_error = data["measured_yield"] - data["measured_yield"]
    calculated_yield_error = data["calculated_yield"] - data["measured_yield"]
    predicted_yield_error = data["predicted_yield"] - data["measured_yield"]

    axes[index].plot(data.index.get_level_values("date"), measured_yield_error, label="Measured yield error")
    axes[index].plot(data.index.get_level_values("date"), calculated_yield_error, label="Calculated yield error")
    axes[index].plot(data.index.get_level_values("date"), predicted_yield_error, label="Predicted yield error")
    axes[index].set_title(f"ID: {id}")

plt.xticks(rotation=45)
plt.legend()
plt.tight_layout()
plt.show()
fig.savefig("error-plots.png")

In [ ]:
from IPython.display import Image, display

display(Image(filename='./error-plots.png', embed=True))

In [ ]:
import calendar


def calculate_monthly_average(x: pd.Series):
    date = x["date"]

    month = date.month
    year = date.year

    days_in_month = calendar.monthrange(year, month)[1]

    return pd.Series({
        "average_measured_yield": x["measured_yield"] / days_in_month,
        "average_calculated_yield": x["calculated_yield"] / days_in_month,
        "average_predicted_yield": x["predicted_yield"] / days_in_month,
    })


grouped_df = dataset.groupby([pd.Grouper(level='id'), pd.Grouper(level='date', freq='M')]).sum().reset_index()

selected_id = 70744


selected_dataset = grouped_df[grouped_df["id"] == selected_id]

average_yields = selected_dataset.apply(
    lambda x: calculate_monthly_average(
       x
    ), axis=1
)

selected_dataset = pd.concat([selected_dataset, average_yields], axis=1)

In [ ]:
plt.style.use("default")

font = {
    'weight': 'bold',
    'size' : 22
}

plt.rc('font', **font)

fig, ax = plt.subplots(figsize=(40, 15))

ax.fill_between(selected_dataset["date"], selected_dataset["average_measured_yield"], alpha=0.3)

ax.plot(selected_dataset["date"], selected_dataset["average_calculated_yield"], label="Calculated yield", color="red", linewidth=4)

ax.plot(selected_dataset["date"], selected_dataset["average_predicted_yield"], label="Predicted yield", color="green", linewidth=4)

plt.xticks(rotation=45)

plt.xlabel("Date")
plt.ylabel("Average monthly yield (kWh)")
plt.title(f"Monthly average yield for installation ID: {selected_id}")
plt.legend()
plt.show()
fig.savefig("grouped.png")


In [ ]:
from IPython.display import Image, display

display(Image(filename='./grouped.png', embed=True))